# Project 1: Calculate a Hessian Matrix by Finite Differences


In this project, you will calculate the second derivative or "Hessian" matrix of
the electronic energy of a molecule with respect to the positions of its atoms.
$$
    \mathbf{H} = 
    \begin{pmatrix}
        H_{11} &
        H_{12} &
        \cdots \\
        H_{21} &
        H_{22} &
        \cdots \\
        \vdots & \vdots & \ddots
    \end{pmatrix} =
    \begin{pmatrix}
        \frac{\partial^2 E}{\partial q_1^2} &
        \frac{\partial^2 E}{\partial q_1 \partial q_2} &
        \cdots \\
        \frac{\partial^2 E}{\partial q_2 \partial q_1} &
        \frac{\partial^2 E}{\partial q_2^2} &
        \cdots \\
        \vdots & \vdots & \ddots
    \end{pmatrix}
$$
Here, $q_1, q_2, ..., q_{3n}$ are elements of a vector containing atomic position coordinates of a molecule with $n$ atoms, $\mathbf{q}=(x_1, y_1, z_1, ..., x_n, y_n, z_n)$.

In future projects, we will use the Hessian matrix to calculate the possible vibrational states of the molecule.

### Formulas

Derivatives of a function can be determined numerically by shifting its
arguments by a small amount $d$ and analyzing the change in the function's value.
This is known as differentation "by finite differences".
The beauty of this technique is that it can be applied even to functions without
a known formula, such as the electronic energy of a molecule.

To calculate the Hessian, we will use the following finite difference formulas
for its diagonal and off-diagonal elements:
$$
\begin{align*}
    H_{ii}
    =
    \frac{\partial^2 E}{\partial q_i^2}
    =&
    \frac{1}{d^2}
    (E(q_i+d) + E(q_i-d) - 2E(q_i))
    \\
    H_{ij}
    =
    \frac{\partial^2 E}{\partial q_i \partial q_j}
    =&
    \frac{1}{2d^2}
    (
    E(q_i+d, q_j+d) + E(q_i-d, e_j-d) - E(q_i+d,q_j) - E(q_i-d,q_j) \\
    &- E(q_i,q_j+d) - E(q_i,q_j-d) + 2E(q_i)
    )
\end{align*}
$$
These formulas are exact in the limit $d\rightarrow0$.
For finite displacements, the error grows as $d^2$, so it is best to choose a
value that is small as possible without introducing noise due to
[floating-point precision errors](https://en.wikipedia.org/wiki/Floating-point_error_mitigation).

### Step 1. Generate initial structure

Some code has been set up for you to generate a chemically reasonable structure for
a given molecule using [SMILES notation](https://en.wikipedia.org/wiki/Simplified_Molecular_Input_Line_Entry_System#Examples).

The SMILES for ethyl radical is given to you. This structure is generated by the
[protochem](../../src/protochem/__init__.py) module in this repository using
[RDKit](https://www.rdkit.org/docs/GettingStartedInPython.html).

In [1]:
from protochem import smiles, geom
from protochem import qc

smi = "C[CH2]"
geo = smiles.geometry(smi)

# Visualize the molecule
print("symbols:", geom.symbols(geo))
print("coordinates:", geom.coordinates(geo))
geom.display(geo)

symbols: ['C', 'C', 'H', 'H', 'H', 'H', 'H']
coordinates: [[-1.15183444e+00  1.58276749e-03 -8.91999558e-03]
 [ 1.63023378e+00 -1.41875685e-01 -5.63693033e-01]
 [-2.10893957e+00  9.31392854e-01 -1.65090989e+00]
 [-1.48788168e+00  1.22589432e+00  1.67560268e+00]
 [-1.99239882e+00 -1.88989478e+00  3.33461032e-01]
 [ 2.53284069e+00  1.66509626e+00 -5.15261351e-02]
 [ 2.57798004e+00 -1.79219573e+00  2.65985341e-01]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Step 2: Optimize equilibrium structure

To calculate electronic properties of the molecule, we will make use of the QC suite of programs from Colton Hicks:
- [qcio](https://github.com/coltonbh/qcio) - Standardizes inputs and outputs to quantum chemistry programs. See [documentation](https://qcio.coltonhicks.com/).
- [qcop](https://github.com/coltonbh/qcop) - Runs quantum chemistry programs to calculate results. See [documentation](https://qcop.coltonhicks.com/).

As a first demonstration, let us optimize our structure to a local energy
minimum using the [CREST](https://crest-lab.github.io/crest-docs/) quantum
chemistry program through the QC interface.
This is a necessary first step to calculating vibrational frequencies, which
must be determine at an "equilibrium" or minimum energy structure.

In [2]:
import qcio
import qcop

prog_inp = qcio.ProgramInput(
    structure=qc.struc.from_geometry(geo),
    calctype="optimization",
    model={"method": "gfn2"},
)

prog_out = qcop.compute("crest", prog_inp)
geo = qc.struc.geometry(prog_out.results.final_structure)
print("Geometry:", geo)
print("Energy:", prog_out.results.final_energy)
geom.display(geo)

Geometry: symbols=['C', 'C', 'H', 'H', 'H', 'H', 'H'] coordinates=array([[-1.3444493 ,  0.07734693,  0.28501429],
       [ 1.42709489, -0.02514059,  0.10888902],
       [-2.16383404,  0.98885073, -1.38305941],
       [-1.95172302,  1.22044656,  1.90472607],
       [-2.15783928, -1.79890302,  0.4775778 ],
       [ 2.48753763,  1.67915723, -0.18438213],
       [ 2.43533222, -1.76994564,  0.3088057 ]]) charge=0 spin=1
Energy: -6.7352941698


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
import numpy

def hessian(geo: geom.Geometry, prog_inp: qcio.ProgramInput) -> numpy.ndarray:
    """Calculate the hessian of a geometry by finite differences.

    :param geo: Geometry
    :param prog_inp: QCIO program input
    :return: Hessian (as a 3n x 3n matrix)
    """
    # your code goes here
    pass